<a href="https://colab.research.google.com/github/scottthomaswx/OutlookPlotter/blob/main/Outlook_Plotter_Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Needed Code

In [1]:
# @title
%%capture
!pip install geopandas
!pip install metpy
!pip install matplotlib
!pip install cartopy
!pip install pandas
import geopandas
import metpy
from metpy.plots import PlotGeometry
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from metpy.plots import USCOUNTIES
import matplotlib.patches as mpatches

Times for Outlook Issuances


Day                | SPC Convective                    | SPC Fire | WPC Excessive Rain | CPC 6-10/8-14 Day Outlooks
-------------------|-----------------------------------|---------|-------------------|-------------------------|
Day 1              | 1200 (6z), 1300, 1630, 2000, 0100 | 1200 (4AM), 1700         | 0100, 0830, 16000 | 1600 ET
Day 2              | 0630, 1730                        | 1000, 2000        | 0830 and 2030
Day 3              | 0730, 1930                              | 2200       | 0830 and 2030
Day 4              | 0400 CT                           | 2200 | 0530 and 1730
Day 5              | 0400 CT                           | 2200| 0530 and 1730
Day 6-8            | 0400 CT                           | 2200| 0530 and 1730

In [34]:
# @markdown ### Enter Needed Information
# @markdown ##### Month and Day Must Be 2 Digits

year = '2025' # @param {type:"string"}
month = '04' # @param {type:"string"}
day = '03' # @param {type:"string"}
outlookday = '1' # @param {type:"string"}
time = '1930' # @param {type:"string"}

WestLong = -92 # @param {type:"integer"}
EastLong = -82 # @param {type:"integer"}
NorthLat = 48 # @param {type:"integer"}
SouthLat = 38 # @param {type:"integer"}

variable = 'CPC Realtime Day 6-10 Precipitation' # @param ["SPC Archive SVR D1-2", "SPC Realtime SVR D1-2", "SPC Archive SVR D3", "SPC Realtime SVR D3", "SPC Archive SVR D4-8", "SPC Realtime SVR D4-8", "SPC Realtime Fire D1-2", "SPC Archive Fire D1-2",  "WPC Realtime Excessive Rain", "CPC Realtime Day 6-10 Temperature", "CPC Realtime Day 6-10 Precipitation", "CPC Realtime Day 8-14 Temperature", "CPC Realtime Day 8-14 Precipitation"]

#Pull Data Shapefiles

In [35]:
# @title
if variable == 'CPC Realtime Day 6-10 Temperature':
  urlday = '610'
  urlvar = 'temp'
if variable == 'CPC Realtime Day 6-10 Precipitation':
  urlday = '610'
  urlvar = 'prcp'
if variable == 'CPC Realtime Day 8-14 Temperature':
  urlday = '814'
  urlvar = 'temp'
if variable == 'CPC Realtime Day 8-14 Precipitation':
  urlday = '814'
  urlvar = 'prcp'

if variable == 'SPC Archive SVR D1-2':
  catoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_cat.nolyr.geojson')
  windoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_wind.nolyr.geojson')
  hailoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_hail.nolyr.geojson')
  sighailoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_sighail.nolyr.geojson')
  toroutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_torn.nolyr.geojson')
  sigtoroutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_sigtorn.nolyr.geojson')
  sigwindoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_sigwind.nolyr.geojson')
elif variable == 'SPC Realtime SVR D1-2':
  catoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_cat.nolyr.geojson')
  windoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_wind.lyr.geojson')
  hailoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_hail.lyr.geojson')
  toroutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_torn.lyr.geojson')
  sigwindoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_sigwind.lyr.geojson')
  sighailoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_sighail.lyr.geojson')
  sigtoroutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_sigtorn.lyr.geojson')
elif variable == 'WPC Realtime Excessive Rain':
  catoutlook = geopandas.read_file(f'https://www.wpc.ncep.noaa.gov/exper/eromap/geojson/Day{outlookday}_Latest.geojson')
elif variable == "SPC Realtime SVR D3":
  catoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_cat.nolyr.geojson')
  proboutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_prob.nolyr.geojson')
  sigproboutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/day{outlookday}otlk_sigprob.nolyr.geojson')
elif variable == "SPC Archive SVR D3":
  catoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_cat.nolyr.geojson')
  proboutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_prob.nolyr.geojson')
  sigproboutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/outlook/archive/{year}/day{outlookday}otlk_{year}{month}{day}_{time}_sigprob.nolyr.geojson')
elif variable == 'SPC Realtime SVR D4-8':
  catoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/exper/day4-8/day{outlookday}prob.nolyr.geojson')
elif variable == 'SPC Archive SVR D4-8':
  outlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/exper/day4-8/archive/{year}/day{outlookday}prob_{year}{month}{day}.nolyr.geojson')
elif variable == 'SPC Realtime Fire D1-2':
  rhoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/fire_wx/day{outlookday}fw_windrh.nolyr.geojson')
  drytoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/fire_wx/day{outlookday}fw_dryt.nolyr.geojson')
elif variable == 'SPC Archive Fire D1-2':
  drytoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/fire_wx/{year}/day{outlookday}fw_{year}{month}{day}_{time}_dryt.nolyr.geojson')
  rhoutlook = geopandas.read_file(f'https://www.spc.noaa.gov/products/fire_wx/{year}/day{outlookday}fw_{year}{month}{day}_{time}_windrh.nolyr.geojson')
elif variable == 'CPC Realtime Day 6-10 Temperature' or 'CPC Realtime Day 6-10 Precipitation' or "CPC Realtime Day 8-14 Temperature" or "CPC Realtime Day 8-14 Precipitation":
  df = geopandas.read_file('https://ftp.cpc.ncep.noaa.gov/GIS/us_tempprcpfcst/'+urlday+urlvar+'_latest.zip')

#Create Figure

In [1]:
# @title
if variable == "SPC Realtime SVR D1-2" or variable == "SPC Archive SVR D1-2":
  fig = plt.figure(figsize=(16, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax1 = plt.subplot(2, 2, 1, projection=ccrs.PlateCarree())
  ax1.coastlines(color='black',resolution='10m',zorder=99)
  ax1.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax1.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=8)
  try:
    ax1.add_geometries(catoutlook.geometry[0], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[0], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[1], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[1], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[2], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[2], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[3], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[3], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[4], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[4], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[5], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[5], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[6], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[6], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[7], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[7], edgecolor='black', linewidth=2)
  except:
    pass
  TSTM = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='lightgreen')
  MRGL = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='darkgreen')
  SLGT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='yellow')
  ENHC = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='orange')
  MODT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='red')
  HIGH = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='magenta')
  ax1.legend([TSTM, MRGL, SLGT, ENHC, MODT, HIGH], ['TSTM', 'MRGL', 'SLGT', 'ENH', 'MDT', 'HIGH'],
            loc='upper left', fontsize=10, facecolor='white', markerscale=8, ncol=3, framealpha=1.0,
            labelcolor='k', borderpad=0.7,
            title='  SPC Categorical Outlook', title_fontsize=10).set_zorder(10)
  ax1.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax1.set_box_aspect(0.7)

  ax2 = plt.subplot(2, 2, 2, projection=ccrs.PlateCarree())
  ax2.coastlines(color='black',resolution='10m',zorder=99)
  ax2.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax2.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=8)
  try:
    ax2.add_geometries(toroutlook.geometry[0], crs=ccrs.PlateCarree(), facecolor=toroutlook.fill[0], edgecolor='black', linewidth=2)
    if toroutlook.LABEL[1] != 'SIGN':
      ax2.add_geometries(toroutlook.geometry[1], crs=ccrs.PlateCarree(), facecolor=toroutlook.fill[1], edgecolor='black', linewidth=2)
    if toroutlook.LABEL[2] != 'SIGN':
      ax2.add_geometries(toroutlook.geometry[2], crs=ccrs.PlateCarree(), facecolor=toroutlook.fill[2], edgecolor='black', linewidth=2)
    if toroutlook.LABEL[3] != 'SIGN':
      ax2.add_geometries(toroutlook.geometry[3], crs=ccrs.PlateCarree(), facecolor=toroutlook.fill[3], edgecolor='black', linewidth=2)
    if toroutlook.LABEL[4] != 'SIGN':
      ax2.add_geometries(toroutlook.geometry[4], crs=ccrs.PlateCarree(), facecolor=toroutlook.fill[4], edgecolor='black', linewidth=2)
    if toroutlook.LABEL[5] != 'SIGN':
      ax2.add_geometries(toroutlook.geometry[5], crs=ccrs.PlateCarree(), facecolor=toroutlook.fill[5], edgecolor='black', linewidth=2)
    if toroutlook.LABEL[6] != 'SIGN':
      ax2.add_geometries(toroutlook.geometry[6], crs=ccrs.PlateCarree(), facecolor=toroutlook.fill[6], edgecolor='black', linewidth=2)
    if toroutlook.LABEL[7] != 'SIGN':
      ax2.add_geometries(toroutlook.geometry[7], crs=ccrs.PlateCarree(), facecolor=toroutlook.fill[7], edgecolor='black', linewidth=2)
  except:
    pass
  try:
    ax2.add_geometries(sigtoroutlook.geometry[0], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
    ax2.add_geometries(sigtoroutlook.geometry[1], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
    ax2.add_geometries(sigtoroutlook.geometry[2], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
  except:
    pass
  TWO = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#008b00')
  FIVE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8b4726')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ffc800')
  FIFT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ff0000')
  THRT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ff00ff')
  FRFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#912cee')
  SXTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#104e8b')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')
  ax2.legend([TWO, FIVE, TEN, FIFT, THRT, FRFV, SXTY, SIG], ['2%', '5%', '10%', '15%', '30%', '45%', '60%', 'SIG'],
            loc='upper left', fontsize=10, facecolor='white', markerscale=8, ncol=3, framealpha=1.0,
            labelcolor='k', borderpad=0.7,
            title='  Tornado Probability', title_fontsize=10).set_zorder(10)
  ax2.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax2.set_box_aspect(0.7)

  ax3 = plt.subplot(2, 2, 3, projection=ccrs.PlateCarree())
  ax3.coastlines(color='black',resolution='10m',zorder=99)
  ax3.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax3.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=8)
  try:
    ax3.add_geometries(windoutlook.geometry[0], crs=ccrs.PlateCarree(), facecolor=windoutlook.fill[0], edgecolor='black', linewidth=2)
    if windoutlook.LABEL[1] != 'SIGN':
      ax3.add_geometries(windoutlook.geometry[1], crs=ccrs.PlateCarree(), facecolor=windoutlook.fill[1], edgecolor='black', linewidth=2)
    if windoutlook.LABEL[2] != 'SIGN':
      ax3.add_geometries(windoutlook.geometry[2], crs=ccrs.PlateCarree(), facecolor=windoutlook.fill[2], edgecolor='black', linewidth=2)
    if windoutlook.LABEL[3] != 'SIGN':
      ax3.add_geometries(windoutlook.geometry[3], crs=ccrs.PlateCarree(), facecolor=windoutlook.fill[3], edgecolor='black', linewidth=2)
    if windoutlook.LABEL[4] != 'SIGN':
      ax3.add_geometries(windoutlook.geometry[4], crs=ccrs.PlateCarree(), facecolor=windoutlook.fill[4], edgecolor='black', linewidth=2)
    if windoutlook.LABEL[5] != 'SIGN':
      ax3.add_geometries(windoutlook.geometry[5], crs=ccrs.PlateCarree(), facecolor=windoutlook.fill[5], edgecolor='black', linewidth=2)
    if windoutlook.LABEL[6] != 'SIGN':
      ax3.add_geometries(windoutlook.geometry[6], crs=ccrs.PlateCarree(), facecolor=windoutlook.fill[6], edgecolor='black', linewidth=2)
    if windoutlook.LABEL[7] != 'SIGN':
      ax3.add_geometries(windoutlook.geometry[7], crs=ccrs.PlateCarree(), facecolor=windoutlook.fill[7], edgecolor='black', linewidth=2)
  except:
    pass
  try:
    ax3.add_geometries(sigwindoutlook.geometry[0], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
    ax3.add_geometries(sigwindoutlook.geometry[1], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
    ax3.add_geometries(sigwindoutlook.geometry[2], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
  except:
    pass
  FIVE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8b4726')
  FIFT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ffc800')
  THRT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ff0000')
  FRFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ff00ff')
  SXTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#912cee')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')
  ax3.legend([FIVE, FIFT, THRT, FRFV, SXTY, SIG], ['5%', '15%', '30%', '45%', '60%', 'SIG'],
            loc='upper left', fontsize=10, facecolor='white', markerscale=8, ncol=3, framealpha=1.0,
            labelcolor='k', borderpad=0.7,
            title='  Severe Wind Probability', title_fontsize=10).set_zorder(10)
  ax3.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax3.set_box_aspect(0.7)

  ax4 = plt.subplot(2, 2, 4, projection=ccrs.PlateCarree())
  ax4.coastlines(color='black',resolution='10m',zorder=99)
  ax4.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax4.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=8)
  try:
    ax4.add_geometries(hailoutlook.geometry[0], crs=ccrs.PlateCarree(), facecolor=hailoutlook.fill[0], edgecolor='black', linewidth=2)
    if hailoutlook.LABEL[1] != 'SIGN':
      ax4.add_geometries(hailoutlook.geometry[1], crs=ccrs.PlateCarree(), facecolor=hailoutlook.fill[1], edgecolor='black', linewidth=2)
    if hailoutlook.LABEL[2] != 'SIGN':
      ax4.add_geometries(hailoutlook.geometry[2], crs=ccrs.PlateCarree(), facecolor=hailoutlook.fill[2], edgecolor='black', linewidth=2)
    if hailoutlook.LABEL[3] != 'SIGN':
      ax4.add_geometries(hailoutlook.geometry[3], crs=ccrs.PlateCarree(), facecolor=hailoutlook.fill[3], edgecolor='black', linewidth=2)
    if hailoutlook.LABEL[4] != 'SIGN':
      ax4.add_geometries(hailoutlook.geometry[4], crs=ccrs.PlateCarree(), facecolor=hailoutlook.fill[4], edgecolor='black', linewidth=2)
    if hailoutlook.LABEL[5] != 'SIGN':
      ax4.add_geometries(hailoutlook.geometry[5], crs=ccrs.PlateCarree(), facecolor=hailoutlook.fill[5], edgecolor='black', linewidth=2)
    if hailoutlook.LABEL[6] != 'SIGN':
      ax4.add_geometries(hailoutlook.geometry[6], crs=ccrs.PlateCarree(), facecolor=hailoutlook.fill[6], edgecolor='black', linewidth=2)
    if hailoutlook.LABEL[7] != 'SIGN':
      ax4.add_geometries(hailoutlook.geometry[7], crs=ccrs.PlateCarree(), facecolor=hailoutlook.fill[7], edgecolor='black', linewidth=2)
  except:
    pass
  try:
    ax4.add_geometries(sighailoutlook.geometry[0], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
    ax4.add_geometries(sighailoutlook.geometry[1], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
    ax4.add_geometries(sighailoutlook.geometry[2], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
  except:
    pass
  FIVE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8b4726')
  FIFT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ffc800')
  THRT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ff0000')
  FRFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ff00ff')
  SXTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#912cee')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')
  ax4.legend([FIVE, FIFT, THRT, FRFV, SXTY, SIG], ['5%', '15%', '30%', '45%', '60%', 'SIG'],
            loc='upper left', fontsize=10, facecolor='white', markerscale=8, ncol=3, framealpha=1.0,
            labelcolor='k', borderpad=0.7,
            title='  Severe Hail Probability', title_fontsize=10).set_zorder(10)
  ax4.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax4.set_box_aspect(0.7)

  plt.figtext(0.50, 0.88, f'SPC Day {outlookday} Convective Outlook Issued {time}z {month}-{day}-{year} ', ha='center', weight='bold', fontsize=20)
  plt.figtext(0.20, 0.08, f'SCOTT THOMAS | SOURCE: SPC', weight='bold', color='black', fontsize=13, ha='center')
  plt.savefig(f'SPC_D{outlookday}_ConvectiveOutlook_{year}_{month}_{day}_{time}.png', bbox_inches = 'tight')

if variable == "WPC Realtime Excessive Rain":
  fig = plt.figure(figsize=(16, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax1 = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
  ax1.coastlines(color='black',resolution='10m',zorder=99)
  ax1.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax1.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=8)
  i = catoutlook['OUTLOOK']
  color = []
  q = 0
  while q < len(i):
      if i[q] == 'Marginal (At Least 5%)':
          color.append('Green')
      elif i[q] == 'Slight (At Least 15%)':
          color.append('Yellow')
      elif i[q] == 'Moderate (At Least 40%)':
          color.append('Red')
      elif i[q] == 'High (At Least 70%)':
          color.append('deeppink')
      q+=1
  catoutlook['fill'] = color
  try:
    ax1.add_geometries(catoutlook.geometry[0], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[0], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[1], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[1], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[2], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[2], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[3], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[3], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[4], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[4], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[5], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[5], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[6], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[6], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[7], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[7], edgecolor='black', linewidth=2)
  except:
    pass
  MRGL = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='darkgreen')
  SLGT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='yellow')
  MODT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='red')
  HIGH = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='magenta')
  ax1.legend([MRGL, SLGT, MODT, HIGH], ['MRGL', 'SLGT', 'MDT', 'HIGH'],
            loc='upper left', fontsize=12, facecolor='white', markerscale=12, ncol=3, framealpha=1.0,
            labelcolor='k', borderpad=0.7,
            title=' Excessive Rainfall Outlook', title_fontsize=12).set_zorder(10)
  ax1.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  plt.figtext(0.50, 0.88, f'WPC Day {outlookday} Excessive Rainfall Outlook Issued {time}z {month}-{day}-{year} ', ha='center', weight='bold', fontsize=20)
  plt.figtext(0.22, 0.10, f'SCOTT THOMAS | SOURCE: WPC', weight='bold', color='black', fontsize=13, ha='center')
  plt.savefig(f'WPC_D{outlookday}_ExcessiveRainfallOutlook_{year}_{month}_{day}_{time}.png', bbox_inches = 'tight')
  ax1.set_box_aspect(0.7)

if variable == "SPC Realtime SVR D3" or variable == "SPC Archive SVR D3":
  fig = plt.figure(figsize=(16, 8), facecolor='white', edgecolor="black", linewidth =6)
  ax1 = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree())
  ax1.coastlines(color='black',resolution='10m',zorder=99)
  ax1.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax1.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=8)
  try:
    ax1.add_geometries(catoutlook.geometry[0], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[0], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[1], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[1], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[2], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[2], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[3], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[3], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[4], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[4], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[5], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[5], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[6], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[6], edgecolor='black', linewidth=2)
    ax1.add_geometries(catoutlook.geometry[7], crs=ccrs.PlateCarree(), facecolor=catoutlook.fill[7], edgecolor='black', linewidth=2)
  except:
    pass
  TSTM = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='lightgreen')
  MRGL = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='darkgreen')
  SLGT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='yellow')
  ENHC = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='orange')
  MODT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='red')
  ax1.legend([TSTM, MRGL, SLGT, ENHC, MODT], ['TSTM', 'MRGL', 'SLGT', 'ENH', 'MDT'],
            loc='upper left', fontsize=10, facecolor='white', markerscale=8, ncol=3, framealpha=1.0,
            labelcolor='k', borderpad=0.7,
            title='  SPC Categorical Outlook', title_fontsize=10).set_zorder(10)
  ax1.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax1.set_box_aspect(0.7)

  ax2 = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
  ax2.coastlines(color='black',resolution='10m',zorder=99)
  ax2.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax2.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=8)
  try:
    ax2.add_geometries(proboutlook.geometry[0], crs=ccrs.PlateCarree(), facecolor=proboutlook.fill[0], edgecolor='black', linewidth=2)
    if windoutlook.LABEL[1] != 'SIGN':
      ax2.add_geometries(proboutlook.geometry[1], crs=ccrs.PlateCarree(), facecolor=proboutlook.fill[1], edgecolor='black', linewidth=2)
    if proboutlook.LABEL[2] != 'SIGN':
      ax2.add_geometries(proboutlook.geometry[2], crs=ccrs.PlateCarree(), facecolor=proboutlook.fill[2], edgecolor='black', linewidth=2)
    if proboutlook.LABEL[3] != 'SIGN':
      ax2.add_geometries(proboutlook.geometry[3], crs=ccrs.PlateCarree(), facecolor=proboutlook.fill[3], edgecolor='black', linewidth=2)
    if proboutlook.LABEL[4] != 'SIGN':
      ax2.add_geometries(proboutlook.geometry[4], crs=ccrs.PlateCarree(), facecolor=proboutlook.fill[4], edgecolor='black', linewidth=2)
    if proboutlook.LABEL[5] != 'SIGN':
      ax2.add_geometries(proboutlook.geometry[5], crs=ccrs.PlateCarree(), facecolor=proboutlook.fill[5], edgecolor='black', linewidth=2)
    if proboutlook.LABEL[6] != 'SIGN':
      ax2.add_geometries(proboutlook.geometry[6], crs=ccrs.PlateCarree(), facecolor=proboutlook.fill[6], edgecolor='black', linewidth=2)
    if proboutlook.LABEL[7] != 'SIGN':
      ax2.add_geometries(proboutlook.geometry[7], crs=ccrs.PlateCarree(), facecolor=proboutlook.fill[7], edgecolor='black', linewidth=2)
  except:
    pass
  try:
    ax2.add_geometries(sigproboutlook.geometry[0], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
    ax2.add_geometries(sigproboutlook.geometry[1], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
    ax2.add_geometries(sigproboutlook.geometry[2], hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree())
  except:
    pass
  FIVE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8b4726')
  FIFT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ffc800')
  THRT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ff0000')
  FRFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#ff00ff')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')
  ax2.legend([FIVE, FIFT, THRT, FRFV, SIG], ['5%', '15%', '30%', '45%', 'SIG'],
            loc='upper left', fontsize=10, facecolor='white', markerscale=8, ncol=3, framealpha=1.0,
            labelcolor='k', borderpad=0.7,
            title='  Severe Weather Probability', title_fontsize=10).set_zorder(10)
  ax2.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax2.set_box_aspect(0.7)
  plt.figtext(0.50, 0.77, f'SPC Day {outlookday} Convective Outlook Issued {time}z {month}-{day}-{year} ', ha='center', weight='bold', fontsize=20)
  plt.figtext(0.22, 0.20, f'SCOTT THOMAS | SOURCE: SPC', weight='bold', color='black', fontsize=13, ha='center')
  plt.savefig(f'SPC_D{outlookday}_ConvectiveOutlook_{year}_{month}_{day}_{time}.png', bbox_inches = 'tight')

if variable == "SPC Realtime SVR D4-8" or variable == "SPC Archive SVR D4-8":
  fig = plt.figure(figsize=(16, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax1 = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
  ax1.coastlines(color='black',resolution='10m',zorder=99)
  ax1.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax1.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=8)
  try:
    ax1.add_geometries(outlook.geometry[0], crs=ccrs.PlateCarree(), facecolor=outlook.fill[0], edgecolor='black', linewidth=2)
    ax1.add_geometries(outlook.geometry[1], crs=ccrs.PlateCarree(), facecolor=outlook.fill[1], edgecolor='black', linewidth=2)
    ax1.add_geometries(outlook.geometry[2], crs=ccrs.PlateCarree(), facecolor=outlook.fill[2], edgecolor='black', linewidth=2)
    ax1.add_geometries(outlook.geometry[3], crs=ccrs.PlateCarree(), facecolor=outlook.fill[3], edgecolor='black', linewidth=2)
    ax1.add_geometries(outlook.geometry[4], crs=ccrs.PlateCarree(), facecolor=outlook.fill[4], edgecolor='black', linewidth=2)
    ax1.add_geometries(outlook.geometry[5], crs=ccrs.PlateCarree(), facecolor=outlook.fill[5], edgecolor='black', linewidth=2)
    ax1.add_geometries(outlook.geometry[6], crs=ccrs.PlateCarree(), facecolor=outlook.fill[6], edgecolor='black', linewidth=2)
    ax1.add_geometries(outlook.geometry[7], crs=ccrs.PlateCarree(), facecolor=outlook.fill[7], edgecolor='black', linewidth=2)
  except:
    pass
  SLGT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='yellow')
  ENHC = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='orange')
  ax1.legend([SLGT, ENHC], ['15%', '30%'],
            loc='upper left', fontsize=15, facecolor='white', markerscale=15, ncol=3, framealpha=1.0,
            labelcolor='k', borderpad=0.7,
            title='  SPC Categorical Outlook', title_fontsize=15).set_zorder(10)
  ax1.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  plt.figtext(0.50, 0.88, f'SPC Day {outlookday} Convective Outlook Issued {time}z {month}-{day}-{year} ', ha='center', weight='bold', fontsize=20)
  plt.figtext(0.22, 0.10, f'SCOTT THOMAS | SOURCE: SPC', weight='bold', color='black', fontsize=13, ha='center')
  plt.savefig(f'SPC_D{outlookday}_ConvectiveOutlook_{year}_{month}_{day}_{time}.png', bbox_inches = 'tight')

if variable == "SPC Realtime Fire D1-2" or variable == "SPC Archive Fire D1-2":
  fig = plt.figure(figsize=(16, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax1 = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
  ax1.coastlines(color='black',resolution='10m',zorder=99)
  ax1.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax1.add_feature(USCOUNTIES, facecolor='None', edgecolor='gray', zorder=8)
  try:
    ax1.add_geometries(rhoutlook.geometry[0], crs=ccrs.PlateCarree(), facecolor=rhoutlook.fill[0], edgecolor='black', linewidth=2)
    ax1.add_geometries(rhoutlook.geometry[1], crs=ccrs.PlateCarree(), facecolor=rhoutlook.fill[1], edgecolor='black', linewidth=2)
    ax1.add_geometries(rhoutlook.geometry[2], crs=ccrs.PlateCarree(), facecolor=rhoutlook.fill[2], edgecolor='black', linewidth=2)
    ax1.add_geometries(rhoutlook.geometry[3], crs=ccrs.PlateCarree(), facecolor=rhoutlook.fill[3], edgecolor='black', linewidth=2)
    ax1.add_geometries(rhoutlook.geometry[4], crs=ccrs.PlateCarree(), facecolor=rhoutlook.fill[4], edgecolor='black', linewidth=2)
    ax1.add_geometries(rhoutlook.geometry[5], crs=ccrs.PlateCarree(), facecolor=rhoutlook.fill[5], edgecolor='black', linewidth=2)
    ax1.add_geometries(rhoutlook.geometry[6], crs=ccrs.PlateCarree(), facecolor=rhoutlook.fill[6], edgecolor='black', linewidth=2)
    ax1.add_geometries(rhoutlook.geometry[7], crs=ccrs.PlateCarree(), facecolor=rhoutlook.fill[7], edgecolor='black', linewidth=2)
  except:
    pass
  try:
      ax1.add_geometries(drytoutlook.geometry[0],  hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree(), edgecolor =drytoutlook.fill[0])
      ax1.add_geometries(drytoutlook.geometry[1],  hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree(), edgecolor =drytoutlook.fill[1])
      ax1.add_geometries(drytoutlook.geometry[2],  hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree(), edgecolor =drytoutlook.fill[2])
      ax1.add_geometries(drytoutlook.geometry[3],  hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree(), edgecolor =drytoutlook.fill[3])
      ax1.add_geometries(drytoutlook.geometry[4],  hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree(), edgecolor =drytoutlook.fill[4])
      ax1.add_geometries(drytoutlook.geometry[5],  hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree(), edgecolor =drytoutlook.fill[5])
      ax1.add_geometries(drytoutlook.geometry[6],  hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree(), edgecolor =drytoutlook.fill[6])
      ax1.add_geometries(drytoutlook.geometry[7],  hatch='//', facecolor="none", linewidth=2, alpha=1, zorder=4.5, crs=ccrs.PlateCarree(), edgecolor =drytoutlook.fill[7])
  except:
    pass
  ELEV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='#ffbf7f', facecolor='none')
  CRIT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='#ff7f7f', facecolor='none')
  ECRIT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='#ff7fff', facecolor='none')
  ISOT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='#c5a392', hatch = '////', facecolor = 'none')
  SCTT = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='#ff7f7f', hatch = '////', facecolor = 'none')
  ax1.legend([ELEV, CRIT, ECRIT, ISOT, SCTT], ['Elevated', 'Critical', 'Extremely Critical', 'Isolated Dry Tstms', 'Sct Dry Tstms'],
            loc='upper left', fontsize=15, facecolor='white', markerscale=15, ncol=3, framealpha=1.0,
            labelcolor='k', borderpad=0.7,
            title='  SPC Fire Weather Outlook', title_fontsize=15).set_zorder(10)
  ax1.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax1.set_box_aspect(0.7)
  plt.figtext(0.50, 0.88, f'SPC Day {outlookday} Fire Weather Outlook Issued {time}z {month}-{day}-{year} ', ha='center', weight='bold', fontsize=20)
  plt.figtext(0.22, 0.10, f'SCOTT THOMAS | SOURCE: SPC', weight='bold', color='black', fontsize=13, ha='center')
  plt.savefig(f'SPC_D{outlookday}_FireOutlook_{year}_{month}_{day}_{time}.png', bbox_inches = 'tight')

if variable == 'CPC Realtime Day 6-10 Temperature' or 'CPC Realtime Day 6-10 Precipitation' or "CPC Realtime Day 8-14 Temperature" or "CPC Realtime Day 8-14 Precipitation":
  #Query Different Categories
  Normal = df.query('Cat == "Normal"')
  B30 =  df.query('Cat == "Below" and Prob < 40 and Prob >= 33')
  B40 =  df.query('Cat == "Below" and Prob < 50 and Prob >= 40')
  B50 =  df.query('Cat == "Below" and Prob < 60 and Prob >= 50')
  B60 =  df.query('Cat == "Below" and Prob < 70 and Prob >= 60')
  B70 =  df.query('Cat == "Below" and Prob < 80 and Prob >= 70')
  B80 =  df.query('Cat == "Below" and Prob < 90 and Prob >= 80')
  B90 =  df.query('Cat == "Below" and Prob < 100 and Prob >= 90')
  A30 =  df.query('Cat == "Above" and Prob < 40 and Prob >= 33')
  A40 =  df.query('Cat == "Above" and Prob < 50 and Prob >= 40')
  A50 =  df.query('Cat == "Above" and Prob < 60 and Prob >= 50')
  A60 =  df.query('Cat == "Above" and Prob < 70 and Prob >= 60')
  A70 =  df.query('Cat == "Above" and Prob < 80 and Prob >= 70')
  A80 =  df.query('Cat == "Above" and Prob < 90 and Prob >= 80')
  A90 =  df.query('Cat == "Above" and Prob < 100 and Prob >= 90')

  #Create Figure
  fig = plt.figure(figsize=(16, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax = plt.axes(projection=ccrs.PlateCarree())
  ax.coastlines('50m', edgecolor='black')
  ax.add_feature(cfeat.OCEAN.with_scale('50m'))
  ax.add_feature(cfeat.LAND.with_scale('50m'))
  ax.set_extent([-125, -65, 25, 50], ccrs.PlateCarree())

  #Plot Temperature Contours
  if urlvar == 'temp':
    ax.add_geometries(B30.geometry, crs = ccrs.PlateCarree(), facecolor =  'lightblue', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B40.geometry, crs = ccrs.PlateCarree(), facecolor =  'cornflowerblue', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B50.geometry, crs = ccrs.PlateCarree(), facecolor =  'royalblue', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B60.geometry, crs = ccrs.PlateCarree(), facecolor =  'darkblue', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B70.geometry, crs = ccrs.PlateCarree(), facecolor =  'violet', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B80.geometry, crs = ccrs.PlateCarree(), facecolor =  'darkorchid', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B90.geometry, crs = ccrs.PlateCarree(), facecolor =  'purple', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A30.geometry, crs = ccrs.PlateCarree(), facecolor =  'coral', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A40.geometry, crs = ccrs.PlateCarree(), facecolor =  'darkorange', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A50.geometry, crs = ccrs.PlateCarree(), facecolor =  'orangered', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A60.geometry, crs = ccrs.PlateCarree(), facecolor =  'indianred', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A70.geometry, crs = ccrs.PlateCarree(), facecolor =  'brown', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A80.geometry, crs = ccrs.PlateCarree(), facecolor =  'red', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A90.geometry, crs = ccrs.PlateCarree(), facecolor =  'darkred', edgecolor = 'black', zorder = 998)
    ax.add_geometries(Normal.geometry, crs = ccrs.PlateCarree(), facecolor =  'gray', edgecolor = 'black', zorder = 998)

    B30C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='lightblue')
    B40C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='cornflowerblue')
    B50C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='royalblue')
    B60C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='darkblue')
    B70C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='violet')
    B80C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='darkorchid')
    B90C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='purple')
    A30C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='coral')
    A40C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='darkorange')
    A50C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='orangered')
    A60C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='indianred')
    A70C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='brown')
    A80C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='red')
    A90C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='darkred')
    NormC= mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='gray')
    ax.legend([B30C, B40C, B50C, B60C, B70C, B80C, B90C, A30C, A40C, A50C, A60C, A70C, A80C, A90C, NormC],
      ['33-40 % Chance Below Normal (Leaning Below)', '40-50 % Chance Below Normal (Leaning Below)',
      '50-60 % Chance Below Normal (Likely Below)', '60-70 % Chance Below Normal (Likely Below)',
      '70-80 % Chance Below Normal (Likely Below)%', '80-90 % Chance Below Normal (Likely Below)',
      '90-100 % Chance Below Normal (Likely Below)', '33-40 % Chance Above Normal (Leaning Above)',
      '40-50 % Chance Above Normal (Leaning Above)','50-60 % Chance Above Normal (Likely Above)',
      '60-70 % Chance Above Normal (Likely Above)', '70-80 % Chance Above Normal (Likely Above)',
      '80-90 % Chance Above Normal (Likely Above)','90-100 % Chance Above Normal (Likely Above)','Normal'],
      loc='upper center', bbox_to_anchor=(0.50, -0.10), fontsize=10, facecolor='white', markerscale=8, ncol=2, framealpha=1.0,
      labelcolor='k', borderpad=0.5, title='CPC Temperature Probabilities', title_fontsize=10).set_zorder(1000)

  #Plot Precipitation Contours
  elif urlvar == 'prcp':
    ax.add_geometries(B30.geometry, crs = ccrs.PlateCarree(), facecolor =  'bisque', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B40.geometry, crs = ccrs.PlateCarree(), facecolor =  'goldenrod', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B50.geometry, crs = ccrs.PlateCarree(), facecolor =  'darkgoldenrod', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B60.geometry, crs = ccrs.PlateCarree(), facecolor =  'peru', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B70.geometry, crs = ccrs.PlateCarree(), facecolor =  'sienna', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B80.geometry, crs = ccrs.PlateCarree(), facecolor =  'saddlebrown', edgecolor = 'black', zorder = 998)
    ax.add_geometries(B90.geometry, crs = ccrs.PlateCarree(), facecolor =  'maroon', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A30.geometry, crs = ccrs.PlateCarree(), facecolor =  'palegreen', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A40.geometry, crs = ccrs.PlateCarree(), facecolor =  'yellowgreen', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A50.geometry, crs = ccrs.PlateCarree(), facecolor =  'limegreen', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A60.geometry, crs = ccrs.PlateCarree(), facecolor =  'green', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A70.geometry, crs = ccrs.PlateCarree(), facecolor =  'forestgreen', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A80.geometry, crs = ccrs.PlateCarree(), facecolor =  'seagreen', edgecolor = 'black', zorder = 998)
    ax.add_geometries(A90.geometry, crs = ccrs.PlateCarree(), facecolor =  'darkgreen', edgecolor = 'black', zorder = 998)
    ax.add_geometries(Normal.geometry, crs = ccrs.PlateCarree(), facecolor =  'gray', edgecolor = 'black', zorder = 998)

    B30C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='bisque')
    B40C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='goldenrod')
    B50C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='darkgoldenrod')
    B60C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='peru')
    B70C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='sienna')
    B80C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='saddlebrown')
    B90C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='maroon')
    A30C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='palegreen')
    A40C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='yellowgreen')
    A50C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='limegreen')
    A60C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='green')
    A70C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='forestgreen')
    A80C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='seagreen')
    A90C = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='darkgreen')
    NormC = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='gray')

    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box .height])
    ax.legend([B30C, B40C, B50C, B60C, B70C, B80C, B90C, A30C, A40C, A50C, A60C, A70C, A80C, A90C, NormC],
     ['33-40 % Chance Below Normal (Leaning Below)', '40-50 % Chance Below Normal (Leaning Below)',
      '50-60 % Chance Below Normal (Likely Below)', '60-70 % Chance Below Normal (Likely Below)',
      '70-80 % Chance Below Normal (Likely Below)%', '80-90 % Chance Below Normal (Likely Below)',
      '90-100 % Chance Below Normal (Likely Below)', '33-40 % Chance Above Normal (Leaning Above)',
      '40-50 % Chance Above Normal (Leaning Above)','50-60 % Chance Above Normal (Likely Above)',
      '60-70 % Chance Above Normal (Likely Above)', '70-80 % Chance Above Normal (Likely Above)',
      '80-90 % Chance Above Normal (Likely Above)', '90-100 % Chance Above Normal (Likely Above)','Normal'],
      loc='upper center', bbox_to_anchor=(0.50, -0.10), fontsize=10, facecolor='white', markerscale=8, ncol=2, framealpha=1.0, labelcolor='k', borderpad=0.5,
      title='CPC Precipitation Probabilities', title_fontsize=10).set_zorder(1000)

  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=999, linewidth = 4)
  ax.set_box_aspect(0.7)
  plt.figtext(0.20, 0.87, f'{variable} Outlook Valid {df.Start_Date[0].strftime("%Y-%m-%d")} - {df.End_Date[0].strftime("%Y-%m-%d")} ', ha='left', weight='bold', fontsize=16)
  plt.figtext(0.125, 0.11, f'Outlook Issued: {df.Fcst_Date[0].strftime("%Y-%m-%d")}', weight='bold', color='black', fontsize=13, ha='left')
  plt.figtext(0.125, 0.08, f'SCOTT THOMAS | SOURCE: CPC', weight='bold', color='black', fontsize=13, ha='left')

NameError: name 'variable' is not defined

In [ ]:
# @title
